# Retrieval-Augmented Shopping Assistant - EDA

This notebook explores the ABO dataset for initial insights.

In [2]:
import pandas as pd

In [3]:
# Load ABO image dataset
df_img = pd.read_csv("/kaggle/input/amazon-berkeley-objects/images/metadata/images.csv")  # or csv if applicable

In [6]:
# Preview
df

,image_id,height,width,path
0,010-mllS7JL,106,106,14/14fe8812.jpg
1,01dkn0Gyx0L,122,122,da/daab0cad.jpg
2,01sUPg0387L,111,111,d2/d2daaae9.jpg
3,1168jc-5r1L,186,186,3a/3a4e88e6.jpg
4,11RUV5Fs65L,30,500,d9/d91ab9cf.jpg
...,...,...,...,...
398207,B1zv8OpTkBS,2560,2560,6d/6d49d130.jpg
398208,B1zwflWhPIS,2560,2560,b1/b163e0ea.jpg
398209,C1lf45DhhRS,2560,2560,a1/a116d9d1.jpg
398210,C1pEt6jBLiS,2560,2560,9c/9c3e1158.jpg


In [15]:
df.loc[df['image_id'] == "81iZlv3bjpL"]

,image_id,height,width,path
319529,81iZlv3bjpL,2560,1969,8c/8ccb5859.jpg


In [19]:
# Load ABO image dataset
df_metadata = pd.read_json("/kaggle/input/listing/listings/metadata/listings_0.json", lines = True)  # or csv if applicable

In [22]:
df_metadata.head()

,brand,bullet_point,color,item_id,item_name,model_name,model_number,model_year,product_type,style,...,item_weight,material,fabric_type,color_code,product_description,spin_id,3dmodel_id,pattern,finish_type,item_shape
0,"[{'language_tag': 'nl_NL', 'value': 'find.'}]","[{'language_tag': 'nl_NL', 'value': 'Schoen in...","[{'language_tag': 'nl_NL', 'value': 'Veelkleur...",B06X9STHNG,"[{'language_tag': 'nl_NL', 'value': 'Amazon-me...","[{'language_tag': 'nl_NL', 'value': '37753'}]",[{'value': '12-05-04'}],[{'value': 2017}],[{'value': 'SHOES'}],"[{'language_tag': 'nl_NL', 'value': 'Gesloten-...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'language_tag': 'es_MX', 'value': 'AmazonBas...","[{'language_tag': 'es_MX', 'value': 'White Pow...","[{'language_tag': 'es_MX', 'value': 'White Pow...",B07P8ML82R,"[{'language_tag': 'es_MX', 'value': '22"" Botto...",NaN,[{'value': 'AB5013-R22-10'}],NaN,[{'value': 'HARDWARE'}],"[{'language_tag': 'es_MX', 'value': '10 pares'}]",...,"[{'normalized_value': {'unit': 'pounds', 'valu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'language_tag': 'en_AE', 'value': 'AmazonBas...","[{'language_tag': 'en_AE', 'value': '3D printe...","[{'language_tag': 'en_AE', 'value': 'Transluce...",B07H9GMYXS,"[{'language_tag': 'en_AE', 'value': 'AmazonBas...",NaN,[{'value': 'AMG10528516-10'}],NaN,[{'value': 'MECHANICAL_COMPONENTS'}],"[{'language_tag': 'en_AE', 'value': '1-Pack'}]",...,"[{'normalized_value': {'unit': 'pounds', 'valu...","[{'language_tag': 'en_AE', 'value': 'PETG (Pol...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'language_tag': 'en_GB', 'value': 'Stone & B...",NaN,"[{'language_tag': 'en_GB', 'value': 'Stone Bro...",B07CTPR73M,"[{'language_tag': 'en_GB', 'value': 'Stone & B...",NaN,[{'value': '25020039-01'}],NaN,[{'value': 'SOFA'}],NaN,...,"[{'normalized_value': {'unit': 'pounds', 'valu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'language_tag': 'en_AU', 'value': 'The Fix'}]","[{'language_tag': 'en_AU', 'value': 'Embroider...","[{'language_tag': 'en_AU', 'standardized_value...",B01MTEI8M6,"[{'language_tag': 'en_AU', 'value': 'The Fix A...",NaN,[{'value': '880071Havana Tan'}],[{'value': 2017}],[{'value': 'SHOES'}],"[{'language_tag': 'en_AU', 'value': 'French Lo...",...,"[{'normalized_value': {'unit': 'pounds', 'valu...",NaN,"[{'language_tag': 'en_AU', 'value': 'Suede'}]",[#A67B5B],NaN,NaN,NaN,NaN,NaN,NaN
